In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

In [ ]:


def visualize_optical_flow(video_path):
    cap = cv.VideoCapture(video_path)
    frame_stack = []
    
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to read the video")
        cap.release()
        return

    prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)
    mask = np.zeros_like(first_frame)
    mask[..., 1] = 255  # Set saturation to maximum

    fig, ax = plt.subplots()

    frame_count = 0
    skip_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        if frame_count % 9 == 0:
            frame_stack.append(gray)

        if len(frame_stack) == 8:
            stacked_frames = np.stack(frame_stack, axis=-1)
            flow = cv.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
            mask[..., 0] = angle * 180 / np.pi / 2
            mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
            rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)

            ax.imshow(cv.cvtColor(rgb, cv.COLOR_BGR2RGB))
            display(fig)
            clear_output(wait=True)

            # Remove the oldest frame from the stack
            frame_stack.pop(0)

        prev_gray = gray
        frame_count += 1

    cap.release()
    plt.close()

# Call the function with your video path
video_path = 'your_video.mp4'
visualize_optical_flow(video_path)
